In [ ]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_curve, f1_score, precision_score, recall_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
!sudo pip install imbalanced-learn
seed = 42

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data = pd.read_csv("creditcard.csv")
X = data.drop(columns="Class")
y = data["Class"]
print(X)
print("gigi")
print(y)
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)

def evaluate(y_test, y_pred):
    # precision true positive / (true positive + false positive)
    # recall: true positive  / (true positive + false ne)
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    # f1 = 2 * (precision * recall) / (precision + recall)
    f1 = f1_score(y_test, y_pred)
    #auc_score = auc(precision, recall)
    return precision, recall, f1


            Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V20       V21  \
0       0.462388  0.239599  0.098698  0.363787  ...  0.25141

In [ ]:
from sklearn.neural_network import MLPClassifier

# Create model object
clf = MLPClassifier(hidden_layer_sizes=5,
                    random_state=5,
                    verbose=True,
                    learning_rate_init=0.01)

# Fit data onto the model
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
precision, recall, f1 = evaluate(y_test,y_pred)
print(precision, recall, f1)
# # Import accuracy score 
from sklearn.metrics import accuracy_score

# Calcuate accuracy
accuracy_score(y_test,y_pred)

Iteration 1, loss = 6.03226786
Iteration 2, loss = 2.71832763
Iteration 3, loss = 2.07570652
Iteration 4, loss = 1.47724191
Iteration 5, loss = 1.07709574
Iteration 6, loss = 0.17976952
Iteration 7, loss = 0.09320773
Iteration 8, loss = 0.19680579
Iteration 9, loss = 0.53547377
Iteration 10, loss = 0.64701268
Iteration 11, loss = 0.68074553
Iteration 12, loss = 0.65308827
Iteration 13, loss = 0.68716345
Iteration 14, loss = 0.67670349
Iteration 15, loss = 0.69204456
Iteration 16, loss = 0.68365533
Iteration 17, loss = 0.68155567
Iteration 18, loss = 0.67977312
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
[0.5        0.99886256 1.        ] [1.         0.09266131 0.        ] 0.169590266825453


0.5462778959956386

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
!pip install datatable
import datatable as dt
import joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96.9 MB 110 kB/s 


In [ ]:
# Load training data
train_transaction =  dt.fread("train_transaction.csv").to_pandas()
train_identity = dt.fread("train_identity.csv").to_pandas()
# Merge the two files into one training set
trainset = pd.merge(train_transaction, train_identity, on='TransactionID', how='outer')
# Use only selective columns
trainset = trainset[["TransactionID","DeviceType","DeviceInfo","TransactionDT","TransactionAmt","ProductCD","card4","card6","P_emaildomain","R_emaildomain","addr1","addr2","dist1","dist2","isFraud"]]

trainset['TransactionDT_day'] = trainset['TransactionDT'].apply(lambda x: int(x/86400)) # Day count
trainset['TransactionDT_hour'] = trainset['TransactionDT'].apply(lambda x: int(x/86400%1*24)) # Hour of the day
trainset['TransactionDT_min'] = trainset['TransactionDT'].apply(lambda x: int(x/86400%1*24%1*60)) # Minute of the hour
trainset['TransactionDT_sec'] = trainset['TransactionDT'].apply(lambda x: int(x/86400%1*24%1*60%1*60)) # Seconds of the minute

dataset = trainset.copy()

dataset['hr_sin'] = np.sin((dataset['TransactionDT_hour'] + dataset['TransactionDT_min']/60.0)*(np.pi/12.0))
dataset['hr_cos'] = np.cos((dataset['TransactionDT_hour'] + dataset['TransactionDT_min']/60.0)*(np.pi/12.0))
# Scale the Transaction Amount values using Robust Scaler
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()
dataset['TransactionAmt_scaled'] = rob_scaler.fit_transform(dataset['TransactionAmt'].values.reshape(-1,1))
# Perform One hot encoding for categorical values
categorical_cols = ['DeviceType', 'ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain']
ohe = pd.get_dummies(dataset[categorical_cols])
ohe.drop('card6_debit or credit', axis=1, inplace=True)
dataset = dataset.join(ohe)



In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import XGBClassifier

train_cols = ["TransactionAmt_scaled","hr_sin","hr_cos"] + list(ohe.columns)
X, y = dataset[train_cols], dataset['isFraud']

sampler = RandomUnderSampler(random_state=0, sampling_strategy=1.0)

X_resampled, y_resampled = sampler.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)

# Create model object
clf2 = MLPClassifier(hidden_layer_sizes=5,
                    random_state=5,
                    verbose=True,
                    learning_rate_init=0.01)

# Fit data onto the model
clf2.fit(X_train,y_train)
y_pred=clf2.predict(X_test)
precision, recall, f1 = evaluate(y_test,y_pred)

print(metrics.classification_report(y_test, y_pred))

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/ieee-fraud-detection.zip
!unzip gdrive/My\ Drive/creditcard.zip

Archive:  gdrive/My Drive/ieee-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   
Archive:  gdrive/My Drive/creditcard.zip
  inflating: creditcard.csv          


**Torch**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import csv
import xgboost as xgb

# numpy = np.loadtxt("creditcard.csv", dtype = np.float32, delimiter = ';', skiprows = 1)
# columns = next(csv.reader(open("creditcard.csv"), delimiter = ";"))
# pytorch = torch.from_numpy(numpy)

import torch, torch.nn as nn, torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils

class FraudNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 16)
        self.fc2 = nn.Linear(16, 18)
        self.fc3 = nn.Linear(18, 20)
        self.fc4 = nn.Linear(20, 24)
        self.fc5 = nn.Linear(24, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x

# b 

net = FraudNet().double()

# X = data.iloc[:, :-1].values # extracting features
# y = data.iloc[:, -1].values # extracting labels




data = pd.read_csv("creditcard.csv")
X = data.drop(columns="Class")
y = data["Class"]

sc = StandardScaler()
X = sc.fit_transform(X)

# from imblearn.over_sampling import SMOTE
# oversample = SMOTE()
# X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)

X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(np.asarray(y_train)).double()

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

training_epochs = 2
minibatch_size = 64 

train = data_utils.TensorDataset(X_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=minibatch_size, shuffle=True)

for i in range(training_epochs):
    for b, data in enumerate(train_loader, 0):
        inputs, labels = data
        labels = labels.unsqueeze(1)
        y_pred = net(inputs)
        loss = loss_fn(y_pred, labels)
        
        if b % 100:
            print('Epochs: {}, batch: {} loss: {}'.format(i, b, loss))
        #reset gradients
        optimizer.zero_grad()

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()

X_test = torch.from_numpy(np.asarray(X_test))
y_test = torch.from_numpy(np.asarray(y_test)).double()

test = data_utils.TensorDataset(X_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=minibatch_size, shuffle=True)

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.double() == labels).sum().item()

print('Accuracy of the network on the {} inputs: {}'.format(
    X_test.shape[0], 100 * correct/total))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epochs: 0, batch: 1 loss: 0.6866233812955364
Epochs: 0, batch: 2 loss: 0.6804969336868492
Epochs: 0, batch: 3 loss: 0.6780596401553788
Epochs: 0, batch: 4 loss: 0.6724684553757092
Epochs: 0, batch: 5 loss: 0.6675220159859447
Epochs: 0, batch: 6 loss: 0.6608118239837978
Epochs: 0, batch: 7 loss: 0.6552235420733281
Epochs: 0, batch: 8 loss: 0.6499596016802015
Epochs: 0, batch: 9 loss: 0.6428405068754339
Epochs: 0, batch: 10 loss: 0.6369719395208436
Epochs: 0, batch: 11 loss: 0.6327597915604821
Epochs: 0, batch: 12 loss: 0.6259081963965442
Epochs: 0, batch: 13 loss: 0.621804433222112
Epochs: 0, batch: 14 loss: 0.6172109678575919
Epochs: 0, batch: 15 loss: 0.6039463139146274
Epochs: 0, batch: 16 loss: 0.5971928602720341
Epochs: 0, batch: 17 loss: 0.5941004777192599
Epochs: 0, batch: 18 loss: 0.5886919000408095
Epochs: 0, batch: 19 loss: 0.5794889600486037
Epochs: 0, batch: 20 loss: 0.5690923490476593
Epochs: 0, batch: 21 loss: 0.5613863964329321
Epochs: 0, batch: 22 loss: 0.550406906509184

KeyboardInterrupt: ignored

In [ ]:
class FraudNetIEEE(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(141, 16)
        self.fc2 = nn.Linear(16, 18)
        self.fc3 = nn.Linear(18, 20)
        self.fc4 = nn.Linear(20, 24)
        self.fc5 = nn.Linear(24, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)

X_train = torch.from_numpy(np.asarray(X_train))
y_train = torch.from_numpy(np.asarray(y_train)).double()

loss_fn = nn.BCELoss()
netIEEE = FraudNetIEEE().double()
optimizer = torch.optim.Adam(netIEEE.parameters(), lr=1e-3)

training_epochs = 2
minibatch_size = 64 

train = data_utils.TensorDataset(X_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=minibatch_size, shuffle=True)

for i in range(training_epochs):
    for b, data in enumerate(train_loader, 0):
        inputs, labels = data
        labels = labels.unsqueeze(1)
        y_pred = netIEEE(inputs)
        loss = loss_fn(y_pred, labels)
        
        if b % 100:
            print('Epochs: {}, batch: {} loss: {}'.format(i, b, loss))
        #reset gradients
        optimizer.zero_grad()

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()

X_test = torch.from_numpy(np.asarray(X_test))
y_test = torch.from_numpy(np.asarray(y_test)).double()

test = data_utils.TensorDataset(X_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=minibatch_size, shuffle=True)

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = netIEEE(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.double() == labels).sum().item()

print('Accuracy of the network on the {} inputs: {}'.format(
    X_test.shape[0], 100 * correct/total))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epochs: 0, batch: 1 loss: 0.6951972860898588
Epochs: 0, batch: 2 loss: 0.6963628479914058
Epochs: 0, batch: 3 loss: 0.6922631657197525
Epochs: 0, batch: 4 loss: 0.6934685957118327
Epochs: 0, batch: 5 loss: 0.6938734705238205
Epochs: 0, batch: 6 loss: 0.6933966725358037
Epochs: 0, batch: 7 loss: 0.6932327911054825
Epochs: 0, batch: 8 loss: 0.6932818949791287
Epochs: 0, batch: 9 loss: 0.69375092029746
Epochs: 0, batch: 10 loss: 0.6928841737415624
Epochs: 0, batch: 11 loss: 0.6935326133105314
Epochs: 0, batch: 12 loss: 0.6920608679404344
Epochs: 0, batch: 13 loss: 0.6930970265702958
Epochs: 0, batch: 14 loss: 0.6935324796240134
Epochs: 0, batch: 15 loss: 0.6928036147727247
Epochs: 0, batch: 16 loss: 0.6924485484474162
Epochs: 0, batch: 17 loss: 0.6918948191949144
Epochs: 0, batch: 18 loss: 0.6917404021594011
Epochs: 0, batch: 19 loss: 0.6924667008870037
Epochs: 0, batch: 20 loss: 0.6925941425135123
Epochs: 0, batch: 21 loss: 0.6928626435073499
Epochs: 0, batch: 22 loss: 0.6924351597516337

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy of the network on the 56962 inputs: 99.82795547909133
